In [ ]:
# Import Libraries =============================================================
import os
import time
import shutil
import subprocess
import sys
try:
    from google.colab import drive  # type:ignore
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Set Configurations ===========================================================
SAFE_PATH = '/content'
COLAB_PATH = '/content/drive/MyDrive/colab-sync'
REPOSITORY_PARENT_PATH = os.path.join(COLAB_PATH, 'src')
REPOSITORY_PATH = os.path.join(REPOSITORY_PARENT_PATH, 'ml-playverse')
PROJECT_NAMES = ['cifar']
FORCE_MOUNT = False
REUSE_REPO = False
PROCESS_DELAY = 5
PROCESS_TIMEOUT = 25

# Mount Drive ==================================================================
print("\n🔗 Mounting Google Drive on Colab")
if IN_COLAB:
    if FORCE_MOUNT:
        drive.mount("/content/drive", force_remount=True)
    else:
        drive.mount("/content/drive")
print(f"\n⏳ Waiting {PROCESS_DELAY} seconds to finish mounting")
time.sleep(PROCESS_DELAY)
for _ in range(PROCESS_TIMEOUT):
    if os.path.exists(COLAB_PATH):
        break
    time.sleep(1)
else:
    raise TimeoutError("❌ Google Drive mounting timed out")
print("\n✅ Google Drive has been mounted on Colab")

# Delete Parent ================================================================
print("\n🗑️ Deleting old parent directory folder")
if os.path.exists(REPOSITORY_PARENT_PATH):
    if REUSE_REPO:
        print("\n✅ Old parent directory is being reused")
    else:
        shutil.rmtree(REPOSITORY_PARENT_PATH)
        os.chdir(SAFE_PATH)
        print(f"\n⏳ Waiting {PROCESS_DELAY} seconds to finish deleting")
        time.sleep(PROCESS_DELAY)
        for _ in range(PROCESS_TIMEOUT):
            if not os.path.exists(REPOSITORY_PARENT_PATH):
                break
            time.sleep(1)
        else:
            raise TimeoutError("❌ Old parent direcory deletion has timed out")
        print("\n✅ Old parent directory has been deleted")
else:
    print("\n✅ Old parent directory did not exist")

# Create Parent ================================================================
print("\n📁 Parent directory is being created")
if not REUSE_REPO:
    os.makedirs(os.path.dirname(REPOSITORY_PATH), exist_ok=True)
    print(f"\n⏳ Waiting {PROCESS_DELAY} seconds to finish creating")
    time.sleep(PROCESS_DELAY)
    for _ in range(PROCESS_TIMEOUT):
        if os.path.exists(REPOSITORY_PARENT_PATH):
            break
        time.sleep(1)
    else:
        raise TimeoutError("❌ Parent directory loading timed out")
    print("\n✅ Parent directory has been created")
else:
    print("\n✅ Parent directory was already created")

# Clone Repository =============================================================
print("\n📥 Repository is being cloned")
if not REUSE_REPO:
    !git clone https://github.com/saeedkhademidoroh/ml-playverse.git "{REPOSITORY_PATH}"
    print(f"\n⏳ Waiting {PROCESS_DELAY} seconds to finish cloning")
    time.sleep(PROCESS_DELAY)
    for _ in range(PROCESS_TIMEOUT):
        if os.path.isdir(REPOSITORY_PATH) and os.listdir(REPOSITORY_PATH):
            break
        time.sleep(1)
    else:
        raise TimeoutError("❌ Repository cloning timed out")
    print("\n✅ Repository cloned up-to-date")
else:
    print("\n✅ Repository was already cloned")

# Run Project ==================================================================
for project in PROJECT_NAMES:
    project_path = os.path.join(REPOSITORY_PATH, project)
    print(f"\n📂 Working directory is being changed:\n{project_path}")
    if os.path.exists(project_path):
        os.chdir(project_path)
        # Wait for a specified amount of time
        print(f"\n⏳ Waiting {PROCESS_DELAY} seconds to finish changing")
        time.sleep(PROCESS_DELAY)
        print("\n✅ Working directory has been changed")
        print("\n🚀 Running main.py as a subprocess")
        # Execute the script with unbuffered output
        process = subprocess.Popen(
            ["python", "-u", "main.py"],
            cwd=project_path,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True
        )
        # Stream output line-by-line
        for line in process.stdout:
            print(line, end='')
        process.wait()
    else:
        raise FileNotFoundError(f"❌ Working directory not found")